In [ ]:
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
import glob
from kcwi_util_modified import register_sauron_colormap
from kcwi_util_modified import visualization
from kcwi_util_modified import get_datacube
from kcwi_util_modified import ppxf_kinematics_RXJ1131_getGlobal_lens_deredshift
from kcwi_util_modified import remove_quasar_from_galaxy_deredshift
from kcwi_util_modified import find_nearest
from kcwi_util_modified import SN_CaHK
from kcwi_util_modified import select_region
from kcwi_util_modified import voronoi_binning
from kcwi_util_modified import get_voronoi_binning_data
from kcwi_util_modified import get_velocity_dispersion_deredshift
from kcwi_util_modified import kinematics_map
from kcwi_util_modified import stellar_type
from vorbin.voronoi_2d_binning import voronoi_2d_binning
from specim.specfuncs import spec1d
register_sauron_colormap()

In [ ]:
import ppxf

In [ ]:
ppxf.__version__

### The lens spectra is extracted summing the light with a circular aperture of radius 0.5 arcsec. The quasar spectra are from the brightest pixels in image A, B and C.

In [ ]:
libary_dir = 'all_dr2_fits'

#data directory
dir="KCWI_data/"

#KCWI mosaic datacube
name="KCWI_RXJ1131_icubes_mosaic_0.1457"

#spectrum from the lens center
#spectrum_aperture1 = dir + 'lens.fits' # single pixel
spectrum_aperture = dir + 'lens_central_area.fits' # pixels containing an area with radius 0.5 arcsec


#spectrum from the quasar center
quasar_spectrum_A  = dir+'quasar_A.fits'
quasar_spectrum_B  = dir+'quasar_B.fits'
quasar_spectrum_C  = dir+'quasar_C.fits'

#redshift of the lens
z = 0.295
## R=3600. spectral resolution is ~ 1.42 using observed wavelength 5115 (restframe 3950 Ang) 
FWHM_gal = 1.42
FWHM_tem_xshooter = 0.41 ## R=9700 and wavelength 5115/(1+z) or 3949.81 Ang
noise = 0.014 ## initial estimate of the noise
degree = 3 # degree of the additive Legendre polynomial in ppxf
T_exp = 266 * 60 # the exposure time in second for the KCWI datasets.
wave_min = 0.34  # restframe wavelength in micrometer
wave_max = 0.43
velscale_ratio = 2

### Check the spectra

In [ ]:
lens_sp = spec1d.Spec1d(spectrum_aperture, informat='fitsflux', trimsec=[2150, -735])

In [ ]:
fg = plt.figure(figsize=(8, 5))
lens_sp.smooth(1, fig=fg)
lens_sp.mark_lines('abs', z=0.295, usesmooth=True)

In [ ]:
plt.figure(figsize=(12, 6))
hd = fits.open(quasar_spectrum_A)[0].header
wv = (hd['CRVAL1'] + np.arange(hd['NAXIS1'])* hd['CDELT1'])/(1.295) # restframed in lens redshift
plt.plot(wv, fits.open(quasar_spectrum_A)[0].data, label='A')
plt.plot(wv, fits.open(quasar_spectrum_B)[0].data, label="B")
plt.plot(wv, fits.open(quasar_spectrum_C)[0].data, label="C")
plt.legend()

### Create three global lens template from the full librray (628 stellar templates covering the 'UVB' region of around 529 different stars). These three global templates consist of completely non-overlaping set of stellar templates.

### global_temp1 would be used for kinematics measurement and other two global templates are to calculate systematic uncertainty

In [ ]:
temp_all = glob.glob(libary_dir + '/*uvb.fits')

In [ ]:
templates1, pp1, lamRange1, logLam1, lamRange2, logLam2, galaxy1, quasar1 = \
    ppxf_kinematics_RXJ1131_getGlobal_lens_deredshift(libary_dir,
                                                      degree=degree,
                                                      spectrum_aperture=spectrum_aperture,
                                                      wave_min=wave_min,
                                                      wave_max=wave_max,
                                                      velscale_ratio=velscale_ratio,
                                                      z=z,
                                                      noise=noise,
                                                      templates_name='xshooter',
                                                      FWHM=FWHM_gal,
                                                      FWHM_tem=FWHM_tem_xshooter,
                                                      quasar_spectrum=quasar_spectrum_A,
                                                      plot=True, temp_array=temp_all)
nTemplates1 = templates1.shape[1]
global_temp1 = templates1 @ pp1.weights[:nTemplates1]

In [ ]:
m = pp1.weights[:nTemplates1] >0
tmp = np.array(temp_all)[m]
tmp_set_1 = sorted(tmp)
tmp_set_1

In [ ]:
temp_set_new = []
for fl in temp_all:
    if fl in tmp_set_1:
        pass
    else:
        temp_set_new.append(fl)
len(temp_set_new)

In [ ]:
templates2, pp2, lamRange1, logLam1, lamRange2, logLam2, galaxy1, quasar1 = \
    ppxf_kinematics_RXJ1131_getGlobal_lens_deredshift(libary_dir,
                                                      degree=degree,
                                                      spectrum_aperture=spectrum_aperture,
                                                      wave_min=wave_min,
                                                      wave_max=wave_max,
                                                      velscale_ratio=velscale_ratio,
                                                      z=z,
                                                      noise=noise,
                                                      templates_name='xshooter',
                                                      FWHM=FWHM_gal,
                                                      FWHM_tem=FWHM_tem_xshooter,
                                                      quasar_spectrum=quasar_spectrum_A,
                                                      plot=True, temp_array=temp_set_new)
nTemplates2 = templates2.shape[1]
global_temp2 = templates2 @ pp2.weights[:nTemplates2]

In [ ]:
m2 = pp2.weights[:nTemplates2] >0
tmp1 = np.array(temp_set_new)[m2]
sorted(tmp1)

In [ ]:
temp_set_new2 = []
for fl in temp_all:
    if fl in tmp_set_1 or fl in tmp1:
        pass
    else:
        temp_set_new2.append(fl)
len(temp_set_new2)

In [ ]:
templates3, pp3, lamRange1, logLam1, lamRange2, logLam2, galaxy1, quasar1 = \
    ppxf_kinematics_RXJ1131_getGlobal_lens_deredshift(libary_dir,
                                                      degree=degree,
                                                      spectrum_aperture=spectrum_aperture,
                                                      wave_min=wave_min,
                                                      wave_max=wave_max,
                                                      velscale_ratio=velscale_ratio,
                                                      z=z,
                                                      noise=noise,
                                                      templates_name='xshooter',
                                                      FWHM=FWHM_gal,
                                                      FWHM_tem=FWHM_tem_xshooter,
                                                      quasar_spectrum=quasar_spectrum_A,
                                                      plot=True, temp_array=temp_set_new2)
nTemplates3 = templates3.shape[1]
global_temp3 = templates3 @ pp3.weights[:nTemplates3]

In [ ]:
m3 = pp3.weights[:nTemplates3] >0
tmp2 = np.array(temp_set_new2)[m3]
sorted(tmp2)

### Check whether the global templates are completely non-overlaping

In [ ]:
set(tmp_set_1).intersection(set(tmp1)), set(tmp_set_1).intersection(set(tmp2)), set(tmp1).intersection(set(tmp2))

In [ ]:
hdu = fits.open(dir+name+'.fits')
visualization(hdu)

In [ ]:
data_crop = fits.open(dir + name + '_crop.fits')
radius_in_pixels = 21
visualization(data_crop)

### loop over the entire cropped data cube to remove the quasar and obtain the noise cube using global_template_1

In [ ]:
data_no_quasar = np.empty_like(data_crop[0].data)
noise_cube = np.empty_like(data_crop[0].data)

for i in range(radius_in_pixels*2+1):
    for j in range(radius_in_pixels*2+1):
        #print (i, j)
        spectrum_perpixel = data_crop[0].data[:,i,j]
        data_no_quasar[:,i,j], noise_cube[:,i,j], sky = remove_quasar_from_galaxy_deredshift(libary_dir, 
                     degree=degree, spectrum_aperture=spectrum_aperture, wave_min=wave_min,
                     wave_max=wave_max, velscale_ratio=velscale_ratio, quasar_spectrum=quasar_spectrum_A,
                     z=z, noise=noise, templates_name='xshooter', FWHM=FWHM_gal,
                     FWHM_tem=FWHM_tem_xshooter, global_temp=global_temp1, plot=False,
                     spectrum_perpixel=spectrum_perpixel, random_plot=True)

In [ ]:
## store the data if necessary
hdu_noquasar=hdu.copy()
hdu_noquasar[0].data = data_no_quasar
hdu_noquasar.writeto(dir+name+'_noquasar.fits',overwrite=True)

hdu_noise=hdu.copy()
hdu_noise[0].data = noise_cube
hdu_noise.writeto(dir+name+'_noise.fits',overwrite=True)

### Load the quasar contribution removed data and get a SNR map using restframe wavelength range 3985 - 4085 Ang. In the observed wavelength that is 5160 - 5290 Ang. This region is after Ca H&K and before H-delta lines.

In [ ]:
hdu_noquasar = fits.open(dir+name+'_noquasar.fits')
hdu_noise = fits.open(dir+name+'_noise.fits')

data_no_quasar = hdu_noquasar[0].data
noise_cube = hdu_noise[0].data

lin_axis_sky = np.linspace(lamRange1[0], lamRange1[1], data_no_quasar.shape[0])
ind_min = find_nearest(lin_axis_sky, 3985)
ind_max = find_nearest(lin_axis_sky, 4085)
ind_min, ind_max

In [ ]:
SN_per_AA, flux_per_AA, sigma_poisson = SN_CaHK(ind_min, ind_max, data_no_quasar, noise_cube, T_exp)

### Select a region using the SNR map and a chosen radius and apply voronoi binning to the pixels within that region

In [ ]:
sat_x = (4.323 - 4.411) / 0.1457
sat_y = (4.546 - 4.011) / 0.1457

sat_x, sat_y

In [ ]:
sat_x = int(round(sat_x))
sat_y = int(round(sat_y))

sat_x, sat_y

In [ ]:
SN_y_center, SN_x_center = np.unravel_index(SN_per_AA.argmax(), SN_per_AA.shape)

SN_per_AA[SN_y_center+sat_y-1 : SN_y_center+sat_y+1, SN_x_center+sat_x : SN_x_center+sat_x+2] = 0.

# SN_per_AA[SN_y_center, SN_x_center] = 0.


max_radius = 13 # 1.9 arcsec
target_SN = 0.5
print(SN_y_center, SN_x_center)

hdu_full = fits.open(dir+name+'.fits')

origin_imaging_data_perAA = np.mean(hdu_full[0].data[ind_min:ind_max,:,:], axis=0)*2

select_region(dir, origin_imaging_data_perAA, SN_per_AA, SN_x_center, SN_y_center, radius_in_pixels,
              max_radius, target_SN, name)

#### Convert the pixel coordinates in arcsec unit before binning, then perform binning using pixel coordinates in arcsec unit and after binning bring them back to cartesian coordinates.

In [ ]:
x, y, signal, noise = np.loadtxt(dir+'voronoi_2d_binning_'+name+'_input.txt').T

x -= SN_x_center
y -= SN_y_center
x *= 0.1457 # pixel scale of the KCWI data
y *= 0.1457

vor_input = np.vstack((x, y, signal, noise)).T
np.savetxt(dir + 'voronoi_2d_binning_' + name + '_arcsec_input.txt', vor_input,
               fmt=b'%10.6f %10.6f %10.6f %10.6f')

In [ ]:
x, y, signal, noise = np.loadtxt(dir+'voronoi_2d_binning_'+name+'_input.txt').T

for a, b in zip(x, y):
    print(a, b)

### Bin the pixels using target SN 24 and 30

In [ ]:
bin_target_SN = 24
plt.figure(figsize=(8, 8))
voronoi_binning(bin_target_SN, dir, name+'_arcsec')
plt.axhline(20)
plt.tight_layout()
plt.pause(1)

In [ ]:
# convert back to cartesian coordiantes
output = np.loadtxt(dir +'voronoi_2d_binning_' + name+ '_arcsec_output.txt')

x, y, binNum = output.T[0], output.T[1], output.T[2]

x = x / 0.1457 + SN_x_center
y = y / 0.1457 + SN_y_center

np.savetxt(dir+'voronoi_2d_binning_'+ name +'_targetSN_24_output.txt', \
                      np.column_stack([x, y, binNum]),fmt=b'%10.6f %10.6f %8i')

In [ ]:
bin_target_SN = 26
plt.figure(figsize=(8, 8))
voronoi_binning(bin_target_SN, dir, name+'_arcsec')
plt.tight_layout()
plt.pause(1)

In [ ]:
# convert back to cartesian coordiantes
output = np.loadtxt(dir +'voronoi_2d_binning_' + name+ '_arcsec_output.txt')

x, y, binNum = output.T[0], output.T[1], output.T[2]

x = x / 0.1457 + SN_x_center
y = y / 0.1457 + SN_y_center

np.savetxt(dir+'voronoi_2d_binning_'+ name +'_targetSN_26_output.txt', \
                      np.column_stack([x, y, binNum]),fmt=b'%10.6f %10.6f %8i')

### Now bin the data using the voronoi binning map

In [ ]:
get_voronoi_binning_data(dir, name, name+'_targetSN_24')

In [ ]:
get_voronoi_binning_data(dir, name, name+'_targetSN_26')

### Measure the kinematics

In [ ]:
voronoi_binning_data = fits.getdata(dir +'voronoi_binning_' + name + '_targetSN_26' + '_data.fits')

get_velocity_dispersion_deredshift(degree=degree,
                                   spectrum_aperture=spectrum_aperture,
                                   voronoi_binning_data=voronoi_binning_data,
                                   velscale_ratio=velscale_ratio,
                                   z=z, noise=noise, FWHM=FWHM_gal,
                                   FWHM_tem_xshooter=FWHM_tem_xshooter,
                                   dir=dir, libary_dir=libary_dir,
                                   global_temp=global_temp1,
                                   quasar_spectrum=quasar_spectrum_A,
                                   wave_min=wave_min, wave_max=wave_max,
                                   T_exp=T_exp, VD_name='targetSN_24',
                                   plot=False)

In [ ]:
measurements=np.loadtxt(dir + 'VD_%s.txt' % 'targetSN_26')
measurements.shape

### The bins with a velocity dispersion value > 350 or < 150 have been excluded and filled with 'NaN'. The number of those bins are printed below.¶

In [ ]:
for i in range(measurements.shape[0]):
    if measurements[i][1] > 350 or measurements[i][1] < 150:
        print(i) # excluded bins
        measurements[i][0], measurements[i][1] = np.nan, np.nan
        measurements[i][2], measurements[i][3] = np.nan, np.nan
        
np.savetxt(dir + 'VD_%s_excluding_bins.txt' % 'targetSN_26', measurements, fmt='%1.4e')

In [ ]:
## kinematics map after excluding few bins and filling them with nan
VD_2d, dVD_2d, V_2d, dV_2d = kinematics_map(dir, name +'_targetSN_24', radius_in_pixels=21,
                                            VD_name='targetSN_24_excluding_bins', vd_val=1000)

plt.imshow(VD_2d,origin='lower', cmap='sauron', vmax=330, vmin=150)
cbar = plt.colorbar()
cbar.set_label(r'$\sigma$ [km/s]')
plt.show()

In [ ]:
## kinematics for bins with target SN 15
voronoi_binning_data = fits.getdata(dir +'voronoi_binning_' + name + '_targetSN_26' + '_data.fits')

get_velocity_dispersion_deredshift(degree=degree,
                                   spectrum_aperture=spectrum_aperture,
                                   voronoi_binning_data=voronoi_binning_data,
                                   velscale_ratio=velscale_ratio,
                                   z=z, noise=noise, FWHM=FWHM_gal,
                                   FWHM_tem_xshooter=FWHM_tem_xshooter,
                                   dir=dir, libary_dir=libary_dir,
                                   global_temp=global_temp1,
                                   quasar_spectrum=quasar_spectrum_A,
                                   wave_min=wave_min, wave_max=wave_max,
                                   T_exp=T_exp, VD_name='targetSN_26',
                                   plot=False)

In [ ]:
measurements=np.loadtxt(dir + 'VD_%s.txt' % 'targetSN_26')
measurements.shape

### The bins with a velocity dispersion value > 350 or < 150 have been excluded and filled with 'NaN'. The number of those bins are printed below.

In [ ]:
for i in range(measurements.shape[0]):
    if measurements[i][1] > 350 or measurements[i][1] < 150:
        print(i) # excluded bins
        measurements[i][0], measurements[i][1] = np.nan, np.nan
        measurements[i][2], measurements[i][3] = np.nan, np.nan
        
np.savetxt(dir + 'VD_%s_excluding_bins.txt' % 'targetSN_26', measurements, fmt='%1.4e')

In [ ]:
VD_2d, dVD_2d, V_2d, dV_2d = kinematics_map(dir, name+'_targetSN_26', radius_in_pixels=21,
                                            VD_name='targetSN_26_excluding_bins', vd_val=1000)

plt.imshow(VD_2d,origin='lower',cmap='sauron')
cbar = plt.colorbar()
cbar.set_label(r'$\sigma$ [km/s]')
plt.show()

In [ ]:
VD_2d.shape

In [ ]:
bin_list = [0, 5, 20, 46, 144]
voronoi_binning_data = fits.getdata(dir +'voronoi_binning_' + name + '_targetSN_24' + '_data.fits')

for i in bin_list:
    ppxf_kinematics_RXJ1131_getGlobal_lens_deredshift(libary_dir,
                                                      degree=degree,
                                                      spectrum_aperture=spectrum_aperture,
                                                      wave_min=wave_min,
                                                      wave_max=wave_max,
                                                      velscale_ratio=velscale_ratio,
                                                      z=z,
                                                      noise=noise,
                                                      templates_name='xshooter',
                                                      FWHM=FWHM_gal,
                                                      FWHM_tem=FWHM_tem_xshooter,
                                                      quasar_spectrum=quasar_spectrum_A,
                                                      plot=True, global_template_lens=global_temp1,
                                                spectrum_perpixel=voronoi_binning_data[i])
    #global_template_lens=global_temp1,
    print('bin num : %d' %i)
    plt.show()

### Check systematics

In [ ]:
wavelength_list = [['rng_1', 0.34, 0.43], ['rng_2', 0.335, 0.425], ['rng_3', 0.33, 0.42]]
deg_list = [2, 3, 4]
global_temp_list = [global_temp1, global_temp2, global_temp3]
quasar_list = [quasar_spectrum_A, quasar_spectrum_B, quasar_spectrum_C]

In [ ]:
voronoi_binning_data = fits.getdata(dir +'voronoi_binning_' + name + '_targetSN_24' + '_data.fits')

for w in wavelength_list:
    wv_min, wv_max = w[1], w[2]
    for d in deg_list:
        deg = d
        for i, gt in enumerate(global_temp_list):
            global_tmp = gt
            for j, qsp in enumerate(quasar_list):
                quasar_spec = qsp
                qsp_name = chr(j+65)
                
                vd_name = 'wave_%s_deg_%d_global_temp_%d_quasar_sp_%s_SN_24' %(w[0], deg, i+1, qsp_name)
                get_velocity_dispersion_deredshift(degree=deg,
                                   spectrum_aperture=spectrum_aperture,
                                   voronoi_binning_data=voronoi_binning_data,
                                   velscale_ratio=velscale_ratio,
                                   z=z, noise=noise, FWHM=FWHM_gal,
                                   FWHM_tem_xshooter=FWHM_tem_xshooter,
                                   dir=dir, libary_dir=libary_dir,
                                   global_temp=global_tmp,
                                   quasar_spectrum=quasar_spec,
                                   wave_min=wv_min, wave_max=wv_max,
                                   T_exp=T_exp, VD_name=vd_name,
                                   plot=False, verbose=False, quiet=True)
            
                ## just to check
                print('VD_wave_%s_deg_%d_global_temp_%d_quasar_sp_%s' %(w[0], deg, i+1, qsp_name) )
                ppxf_kinematics_RXJ1131_getGlobal_lens_deredshift(libary_dir,
                                                      degree=deg,
                                                      spectrum_aperture=spectrum_aperture,
                                                      wave_min=wv_min, wave_max=wv_max,
                                                      velscale_ratio=velscale_ratio,
                                                      z=z, noise=noise,
                                                      templates_name='xshooter',
                                                      FWHM=FWHM_gal,
                                                      FWHM_tem=FWHM_tem_xshooter,
                                                      quasar_spectrum=quasar_spec,
                                                      plot=True, global_template_lens=global_tmp,
                                                spectrum_perpixel=voronoi_binning_data[0])
                plt.show()

In [ ]:
voronoi_binning_data1 = fits.getdata(dir +'voronoi_binning_' + name + '_targetSN_26' + '_data.fits')

for w in wavelength_list:
    wv_min, wv_max = w[1], w[2]
    for d in deg_list:
        deg = d
        for i, gt in enumerate(global_temp_list):
            global_tmp = gt
            for j, qsp in enumerate(quasar_list):
                quasar_spec = qsp
                qsp_name = chr(j+65)
                
                vd_name = 'wave_%s_deg_%d_global_temp_%d_quasar_sp_%s_SN_26' %(w[0], deg, i+1, qsp_name)
                get_velocity_dispersion_deredshift(degree=deg,
                                   spectrum_aperture=spectrum_aperture,
                                   voronoi_binning_data=voronoi_binning_data1,
                                   velscale_ratio=velscale_ratio,
                                   z=z, noise=noise, FWHM=FWHM_gal,
                                   FWHM_tem_xshooter=FWHM_tem_xshooter,
                                   dir=dir, libary_dir=libary_dir,
                                   global_temp=global_tmp,
                                   quasar_spectrum=quasar_spec,
                                   wave_min=wv_min, wave_max=wv_max,
                                   T_exp=T_exp, VD_name=vd_name,
                                   plot=False, verbose=False, quiet=True)
            
                ## just to check
                print('VD_wave_%s_deg_%d_global_temp_%d_quasar_sp_%s' %(w[0], deg, i+1, qsp_name) )
                ppxf_kinematics_RXJ1131_getGlobal_lens_deredshift(libary_dir,
                                                      degree=deg,
                                                      spectrum_aperture=spectrum_aperture,
                                                      wave_min=wv_min, wave_max=wv_max,
                                                      velscale_ratio=velscale_ratio,
                                                      z=z, noise=noise,
                                                      templates_name='xshooter',
                                                      FWHM=FWHM_gal,
                                                      FWHM_tem=FWHM_tem_xshooter,
                                                      quasar_spectrum=quasar_spec,
                                                      plot=True, global_template_lens=global_tmp,
                                                spectrum_perpixel=voronoi_binning_data1[0])
                plt.show()